In [185]:
import nltk
from nltk.corpus import stopwords
import pandas as pd
import pickle
import numpy as np
from nltk import word_tokenize, FreqDist
import string
from os import path
from PIL import Image
import matplotlib.pyplot as plt
# from wordcloud import WordCloud, STOPWORDS,ImageColorGenerator
# from nltk.sentiment.sentiment_analyzer import SentimentAnalyzer as sa
# from nltk.classify.scikitlearn import SklearnClassifier
# from nltk.classify import ClassifierI
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import spacy
from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import Dataset
from surprise import Reader
reader= Reader(rating_scale=(-5.0, 5.0))
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

with open('userratingsDF.pickle', 'rb') as f:
    dfr = pickle.load(f)
    
with open('CategoricalItemDF.pickle', 'rb') as f:
    full = pickle.load(f)

with open('_SENT&RATINGDF.pickle', 'rb') as f:
    df1 = pickle.load(f)

with open('recdf.pickle', 'rb') as f:
    rec = pickle.load(f)


In [186]:
df1

,content,date,rating,title,url,user,average_content_sentiment,average_title_sentiment,content_sentiment,title_sentiment
0,i have used this product for 4 years. easy to ...,2012-01-30,5,Years,10236357,kimber,0.30640,0.507400,0.213,0.000
1,"the best eye creme ever tried, it delivers res...",2005-09-22,5,simply the best,10236357,george,0.30640,0.507400,0.325,0.677
2,the cream was easy to use and did not feel gre...,2007-12-19,5,Great Results. Loved It.,10236357,mike,0.30640,0.507400,0.425,0.800
3,"i didn't have any wrinkles around my eyes, but...",2010-02-05,5,It's worth the money,10236357,adrian,0.30640,0.507400,0.320,0.388
4,it keeps your eye area moisturizer without bei...,2014-04-21,5,Great eye product,10236357,westtexas,0.30640,0.507400,0.249,0.672
5,i am surprised that jurlique did not name this...,2012-09-27,5,Comforting,10305907,marvymds,0.38100,0.753000,0.259,1.000
6,it seems like such a small tube but fortunatel...,2014-03-10,4,Mattifying; Great for combination skin,10305907,trixie,0.38100,0.753000,0.503,0.506
7,this is the best toner i have ever used! it is...,2006-08-04,5,Love it!,10305956,sandi,0.24850,0.378000,0.326,0.818
8,"this was my first purchse with skinstore, and ...",2008-01-01,5,"Bright, Beautiful and Bearly there",10305956,"cara bryant, newman, western australia",0.24850,0.378000,0.248,0.694
9,just read alcohol is a terrible ingrediant for...,2008-03-01,1,Alcohol?!,10305956,confused,0.24850,0.378000,0.000,0.000


In [124]:
full

,price,prodName,rating,url,age,darkcircles,acne,dry,redness,sensitive,...,moisturizer,balm,oil,mask,peel,lip,eye,supplement,tool,brand
0,25.08,Paula's Choice Skin Perfecting 2% BHA Liquid E...,5.00,11174178,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,The Ordinary
1,67.00,SkinCeuticals Phyto Corrective Gel,4.11,11289629,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,Erno Laszlo
2,105.00,Sunday Riley Good Genes Treatment 30ml,4.73,10915787,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,Slip
3,17.85,Paula's Choice Skin Balancing Pore-Reducing To...,5.00,11174171,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Slip
4,62.00,Dermalogica Special Cleansing Gel 16.9oz,4.50,10364095,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Paula's Choice
5,83.00,SkinCeuticals Hydrating B5 Hyaluronic Acid Gel...,4.68,11289619,1,0,1,1,0,1,...,1,0,0,0,0,0,0,0,0,Epionce
6,98.00,SkinCeuticals Discoloration Defense Dark Spot ...,5.00,11679046,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Erno Laszlo
7,85.00,Slip Silk Pillowcase - Queen - Pink,5.00,11561754,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,Slip
8,249.00,SkinCeuticals Restore and Hydrate Regimen,5.00,11976675,1,0,1,1,0,1,...,1,0,0,0,0,0,0,0,0,ARgENTUM
9,85.00,Slip Silk Pillowcase - Queen - White,4.67,11561755,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,Slip


In [128]:
alldf=dfr[['url','prodName','rating','user']]

In [183]:
alldf.groupby(['url','user','prodName']).mean()

rating
url      user                                   prodName                 
10236357 adrian                                 Anthony                 5
         george                                 Anthony                 5
         kimber                                 Anthony                 5
         mike                                   Anthony                 5
         westtexas                              Anthony                 5
10305907 marvymds                               Jurlique                5
         trixie                                 Jurlique                4
10305956 cara bryant, newman, western australia Elizabeth Arden         5
         confused                               Elizabeth Arden         1
         piddee                                 Elizabeth Arden         5
         sandi                                  Elizabeth Arden         5
10306437 douya                                  Caudalie                4
         lynx                                   Caudalie                5
         msjani                                 Caudalie                5
         shmoovee                               Caudalie                5
10306558 ab                                     Caudalie                1
         believer                               Caudalie                5
         kalliem                                Caudalie                5
         leelee23                               Caudalie                2
         lv25                                   Caudalie                5
         mompars                                Caudalie                5
         mrsoctagon                             Caudalie                5
         nmgiovi                                Caudalie                3
         pammy                                  Caudalie                4
         tonim                                  Caudalie                2
10362821 eman                                   MenScience              5
         thatguy                                MenScience              4
         tonic123                               MenScience              4
10362879 alfie1025                              Peter Thomas Roth       5
         blondebabe                             Peter Thomas Roth       5
...                                                                   ...
11841279 kgator                                 The Ordinary            5
         liaa                                   The Ordinary            5
11848942 ingrid                                 La Roche-Posay          5
11848946 valleygal                              SkinCeuticals           5
11888501 kate                                   SkinCeuticals           5
11907698 lorim                                  SkinCeuticals           5
11976680 andyandy                               SkinCeuticals           5
11986269 nicobluews                             NuFACE                  4
11986270 yue                                    NuFACE                  4
11995332 ollie                                  Pai                     5
12004604 mary                                   Boots No7               5
12004607 erin                                   Boots No7               5
12004608 sally                                  Boots No7               5
12031613 emma                                   SkinCeuticals           5
         nikkib                                 SkinCeuticals           5
12060134 lucille                                Supermood               5
12060141 tiffany                                Supermood               5
12060145 molly sharma                           Supermood               5
12060147 js                                     Supermood               5
12060158 kenz                                   The Perfect V           5
12060161 js                                     The Perfect V           5
12060163 agn                                    The Perfect V          

In [182]:
alldf.sort_values(['url','user'])

,url,prodName,rating,user
1587,10236357,Anthony,5,adrian
1589,10236357,Anthony,5,george
1586,10236357,Anthony,5,kimber
1588,10236357,Anthony,5,mike
1585,10236357,Anthony,5,westtexas
2754,10305907,Jurlique,5,marvymds
2753,10305907,Jurlique,4,trixie
2167,10305956,Elizabeth Arden,5,"cara bryant, newman, western australia"
2166,10305956,Elizabeth Arden,1,confused
2165,10305956,Elizabeth Arden,5,piddee


In [180]:
alldf.sort_values('user')

,url,prodName,rating,user
3247,11289064,PCA SKIN,5,000000
321,11294885,Juice Beauty,2,1908
1426,11287696,Glytone,5,2
1827,11054488,First Aid Beauty,5,24fan
1637,11054488,First Aid Beauty,5,24fan
1014,11291061,Neocutis,5,4strong
2985,10543756,Dr. Brandt,5,61bks
2876,11289657,SkinMedica,5,7asm0
2541,11289657,SkinMedica,5,7asm0
2285,11288446,Koh Gen Do,5,a girl


In [131]:
# data=Dataset.load_from_df(df1[['user','url','rating']],reader=reader)
# data


In [134]:
data=Dataset.load_from_df(alldf[['user','url','rating']],reader=reader)
data

In [187]:
data=Dataset.load_from_df(rec[['user','url','rating']],reader=reader)
data

In [188]:
train, test = train_test_split(data,test_size=0.2)

In [189]:
sim_cos = {'name':'cosine','user_based':False}

In [190]:
basic = knns.KNNBasic(sim_options=sim_cos)
basic.fit(train)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [191]:
basic.sim
predictions = basic.test(test)
print(accuracy.rmse(predictions))

RMSE: 0.9241
0.9241331609581693


In [192]:
sim_cos_user = {'name':'cosine','user_based':True}

In [193]:
basic = knns.KNNBasic(sim_options=sim_cos_user)
basic.fit(train)
basic.sim
predictions = basic.test(test)
print(accuracy.rmse(predictions))

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9245
0.9245380479875069


In [194]:
sim_pearson = {'name':'pearson','user_based':False}
basic_pearson = knns.KNNBasic(sim_options=sim_pearson)
basic_pearson.fit(train)
predictions = basic_pearson.test(test)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9240
0.923979272907225


In [195]:
sim_pearson_user = {'name':'pearson','user_based':True}
basic_pearson = knns.KNNBasic(sim_options=sim_pearson_user)
basic_pearson.fit(train)
predictions = basic_pearson.test(test)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9240
0.923979272907225


In [196]:
sim_pearson = {'name':'pearson','user_based':False}
knn_means = knns.KNNWithMeans(sim_options=sim_pearson)
knn_means.fit(train)
predictions = knn_means.test(test)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9475
0.9475118979558839


In [197]:
sim_pearson_us = {'name':'pearson','user_based':True}
knn_means = knns.KNNWithMeans(sim_options=sim_pearson_us)
knn_means.fit(train)
predictions = knn_means.test(test)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9848
0.9848180930860436


In [198]:
sim_pearson = {'name':'pearson','user_based':False}
knn_baseline = knns.KNNBaseline(sim_options=sim_pearson)
knn_baseline.fit(train)
predictions = knn_baseline.test(test)
print(accuracy.rmse(predictions))

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9230
0.9230331699141838


In [199]:
sim_pearson_use = {'name':'pearson','user_based':True}
knn_baseline = knns.KNNBaseline(sim_options=sim_pearson_use)
knn_baseline.fit(train)
predictions = knn_baseline.test(test)
print(accuracy.rmse(predictions))

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9230
0.9230331699141838


In [200]:
predictions[5][3]

4.219708644929659

In [201]:
from surprise.prediction_algorithms import SVD
from surprise.model_selection import GridSearchCV

In [202]:
param_grid = {'n_factors':[20,100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs_model = GridSearchCV(SVD,param_grid=param_grid,n_jobs = -1,joblib_verbose=5)
gs_model.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  66 out of  80 | elapsed:    2.9s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    3.3s finished


In [203]:
svd = SVD(n_factors=100,n_epochs=10,lr_all=0.005,reg_all=0.4)
svd.fit(train)
predictions = svd.test(test)
print(accuracy.rmse(predictions))

RMSE: 0.9242
0.9241714166225277


In [204]:
user_34_prediction = svd.predict('3','20')
user_34_prediction

Prediction(uid='3', iid='20', r_ui=None, est=4.55314371257485, details={'was_impossible': False})

In [212]:
df=dfr[['user','rating','url','prodName']]

In [213]:
df.sample(1)

,user,rating,url,prodName
832,gorillaz,5,10813821,Dr Dennis Gross Skincare


## FIX!!!!

In [231]:
def skincare_rater(df,num, typ=None, problem=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if typ:
            prod = df[df[''].str.contains(genre)].sample(1)
        else:
            prod = df.sample(1)
        print(prod)
        rating = input('How do you rate this product on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_prod=df.sample(1)[['url','prodName']]
            rating_one_prod['user']=1000
            rating_one_prod['rating']=rating
            ratedict={'url':rating_one_prod['url'],'prodName':rating_one_prod['ur']}
            rating_list.append(rating_one_prod) 
            num -= 1
    return rating_list

In [232]:
skincare_rater(rec,5)

     prodName  rating       url    user
3041     ESPA       5  11553548  sophia
How do you rate this product on a scale of 1-5, press n if you have not seen :
1
              prodName  rating       url   user
954  Peter Thomas Roth       5  10362879  steph
How do you rate this product on a scale of 1-5, press n if you have not seen :
2
         prodName  rating       url user
1084  Erno Laszlo       4  11640752   ba
How do you rate this product on a scale of 1-5, press n if you have not seen :
3
                      prodName  rating       url user
2283  Dermelect Cosmeceuticals       5  11331846  ash
How do you rate this product on a scale of 1-5, press n if you have not seen :
4
     prodName  rating       url     user
1646    Avene       5  11286360  stellaa
How do you rate this product on a scale of 1-5, press n if you have not seen :
5


[          url      prodName  user rating
 518  11363399  The Ordinary  1000      1,
            url      prodName  user rating
 2219  11288161  Juice Beauty  1000      2,
            url    prodName  user rating
 2135  11058748  L'Occitane  1000      3,
          url      prodName  user rating
 29  10915787  Sunday Riley  1000      4,
            url      prodName  user rating
 3044  11288163  Juice Beauty  1000      5]

### 

In [ ]:
new_ratings_df = df.append(user_rating,ignore_index=True)
new_data = Dataset.load_from_df(new_ratings_df,reader)